In [1]:
import numpy as np
import math

import torch
import torch.nn as nn

Задание 1

In [36]:
# без torch
class Object:
    def __init__(self, x1, x2, a0, a1, a2):
        self.x1=x1
        self.x2=x2
        self.alpha0=a0
        self.alpha1=a1
        self.alpha2=a2
    def forward(self): # синус как функция активации
        total = self.alpha0 + self.alpha1*self.x1 + self.alpha2*self.x2
        z = math.sin(total)
        return z, total
    def backword(self, *z_reals): #находим координаты градиента, MSE - функция потерь
        grad1 = sum([2*(self.forward()[0]-i)*math.cos(self.forward()[1])*self.x1
          for i in z_reals])  #dL/dz * dz/da1
        grad2 = sum([2*(self.forward()[0]-i)*math.cos(self.forward()[1])*self.x2
          for i in z_reals])  #dL/dz * dz/da2
        return grad1, grad2


In [37]:
d = Object(1,2,3,4,5)
d.backword(17,17,17,19)

(40.63919807170366, 81.27839614340732)

In [219]:
# forward torch
tensor = torch.tensor([1, 2, 3, 4, 5]).float()
wieghts = nn.Parameter(torch.rand(5), requires_grad=True)

tensor_output = wieghts @ tensor.t()
activation = nn.ReLU()
activation(tensor_output)

tensor(6.8862, grad_fn=<ReluBackward0>)

In [223]:
# forward torch Linear
tensor = torch.tensor([1, 2, 3, 4, 5]).float()

linear = nn.Linear(5, 1, bias=True)
linear(tensor)

tensor([2.9338], grad_fn=<AddBackward0>)

In [229]:
# forward torch Linear & ReLU
tensor = torch.tensor([1, 2, 3, 4, 5]).float()

seq = nn.Sequential(
    
        nn.Linear(5, 1, bias=True),
        nn.ReLU()    
)

seq(tensor)

tensor([1.9761], grad_fn=<ReluBackward0>)

In [230]:
# forward torch Linear & Siqmoid
tensor = torch.tensor([1, 2, 3, 4, 5]).float()

seq = nn.Sequential(
    
        nn.Linear(5, 1, bias=True),
        nn.Sigmoid()    
)

seq(tensor)

tensor([0.0185], grad_fn=<SigmoidBackward0>)

In [396]:
class Object:
    def __init__(self, *x):
        self.x=torch.tensor(x).float()
        
    def forward(self): # синус как функция активации
        xx = self.x
        razmer = list(xx.size())[1]
        seq = nn.Sequential(

        nn.Linear(razmer, 1, bias=True),
        nn.Sigmoid()    
        )
        return seq(self.x)
    
    def backward(self, target):
        xx = self.x
        razmer = list(xx.size())[1]
        seq = nn.Sequential(

        nn.Linear(razmer, 1, bias=True),
        nn.Sigmoid()    
        )

        loss = torch.mean((target - xx) ** 2).float()
        loss.backward()
        return seq[0].weight.grad
   

In [397]:
dd = Object([1,2,3])
dd.forward()

tensor([[0.7732]], grad_fn=<SigmoidBackward0>)

In [398]:
# прошу подсказать, почему внутри функции не работает loss.backward
target = torch.randint(0,10,(3,)).float()
dd.backward(target)

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [241]:
# backword входного слоя, верно понимаю?
predict = seq(tensor)

target = torch.randint(0,10,(5,)).float()
loss = torch.mean((target - predict) ** 2)

loss.backward()
print(seq[0].weight.grad)

tensor([[-0.7294, -1.4588, -2.1882, -2.9177, -3.6471]])


In [242]:
# backword выходного слоя
# вопрос: почему не считается? в теории ведь мы можем найти градиент от функции активации (сигмоиды)
predict = seq(tensor)

target = torch.randint(0,10,(5,)).float()
loss = torch.mean((target - predict) ** 2)

loss.backward()
print(seq[1].weight.grad)

AttributeError: 'Sigmoid' object has no attribute 'weight'

In [248]:
# По тангенсу тоже не получается
tensor = torch.tensor([1, 2, 3, 4, 5]).float()

seq = nn.Sequential(
    
        nn.Linear(5, 1, bias=True),
        nn.Tanh(),
        nn.Sigmoid()    
)

predict = seq(tensor)

target = torch.randint(0,10,(5,)).float()
loss = torch.mean((target - predict) ** 2)

loss.backward()
print(seq[1].weight.grad)

AttributeError: 'Tanh' object has no attribute 'weight'

Задание 2 и 3

In [84]:
class SDGMomentum:
    def __init__(self, momentum, lr, model):
        self.momentum = momentum
        self.lr = lr
        self.velocity = torch.zeros_like(model)
        self.model = model.float()

    def step(self, grad):
        self.velocity = self.momentum * self.velocity - self.lr * grad
        self.model += self.velocity

In [99]:
# найти пересечение с осью x линии y = 2 * x - 7 методом градиентного спуска
# ответ: 3.5

def func(x):
    return (2 * x -7) ** 2
           # (2*x - 7)**4

def grad_f(x):
    return 2 * (2 * x - 7) * 2
           # 8 * (2*x - 7)**3

def solver(init_x):
    x = torch.tensor(init_x)
    g = grad_f(x)
    optim = SDGMomentum(0.9, 0.001, x)
    for i in range(300):
        optim.step(g)
        g = grad_f(optim.model)
    print(optim.model)

In [100]:
solver(100)

tensor(3.5000)


In [120]:
# Адаптивный оптимизатор Adagrad

class Adagrad:
    def __init__(self, lr, model):
        self.lr = lr
        self.accumulated = torch.zeros_like(model).float()
        self.model = model.float()

    def step(self, grad):
        self.accumulated += grad**2
        self.model += -grad*(self.lr/self.accumulated**0.5)

In [121]:
# Оптимизатор Adagrad находит ответ с lr=30 и 100 итерациями гр.спуска из точки x=100

def func(x):
    return (2 * x -7) ** 2
           # (2*x - 7)**4

def grad_f(x):
    return 2 * (2 * x - 7) * 2
           # 8 * (2*x - 7)**3

def solver(init_x):
    x = torch.tensor(init_x)
    g = grad_f(x)
    optim = Adagrad(30.0, x)
    for i in range(100):
        optim.step(g)
        g = grad_f(optim.model)
    print(optim.model)

In [122]:
solver(100)

tensor(3.5000)
